# Supervised learning of SNN
# Preliminary Model0

In [1]:
from keras.datasets import mnist
from brian2 import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline

import time
import random

INFO       Cache size for target "cython": 2052 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "/home/dmitry/.cython/brian_extensions" directory. [brian2]


## Data set

In [2]:
# select 3 arbitrary classes out of 10

classes = [0, 1, 5] #random.sample(range(10), 3)
(X_train, y_train), (X_test, y_test) = mnist.load_data()

img_size = 28*28 # input image size
X_train = X_train.reshape(60000, img_size)
X_test = X_test.reshape(10000, img_size)

# simplified classification
X_train = X_train[(y_train == classes[0]) | (y_train == classes[1]) | (y_train == classes[2])]
y_train = y_train[(y_train == classes[0]) | (y_train == classes[1]) | (y_train == classes[2])]
X_test = X_test[(y_test == classes[0]) | (y_test == classes[1]) | (y_test == classes[2])]
y_test = y_test[(y_test == classes[0]) | (y_test == classes[1]) | (y_test == classes[2])]

# pixel intensity to Hz (255 becoms ~63Hz)
X_train = X_train / 4
X_test = X_test / 4

print(f'classes: {classes}')
X_train.shape, X_test.shape

classes: [0, 1, 5]


((18086, 784), (3007, 784))

## Initial data

In [3]:
n_input = 28*28 # input layer

# Setting 10 neurons per class

# 1-st set - Net0
n_e = 100 # e - excitatory
n_i = n_e # i - inhibitory

v_rest_e = -60.*mV # v - membrane potential
v_reset_e = -65.*mV
v_thresh_e = -52.*mV

v_rest_i = -60.*mV
v_reset_i = -45.*mV
v_thresh_i = -40.*mV

# common data
taupre = 20*ms
taupost = taupre
gmax = .05 #.01
dApre = .01
dApost = -dApre * taupre / taupost * 1.05
dApost *= gmax
dApre *= gmax

wi = 3 # strength of inhibitory synapse in Net0

# Apre and Apost - presynaptic and postsynaptic traces, lr - learning rate

stdp0='''w : 1
    lr : 1
    dApre/dt = -Apre / taupre : 1 (event-driven)
    dApost/dt = -Apost / taupost : 1 (event-driven)'''

pre0='''ge += w
    Apre += dApre
    w = clip(w + lr*Apost, 0, gmax)'''

post0='''Apost += dApost
    w = clip(w + lr*Apre, 0, gmax)'''

## Model

In [4]:
class Model0():

    def __init__(self):
        app0 = {}
        
        # input images as rate encoded Poisson generators
        app0['PG'] = PoissonGroup(n_input, rates=np.zeros(n_input)*Hz, name='PG')
        
        # excitatory group
        neuron_e = '''
            dv/dt = (ge*(0*mV-v) + gi*(-100*mV-v) + (v_rest_e-v)) / (100*ms) : volt
            dge/dt = -ge / (5*ms) : 1
            dgi/dt = -gi / (10*ms) : 1
            '''
        app0['EG'] = NeuronGroup(n_e, neuron_e, threshold='v>v_thresh_e', 
                                refractory=5*ms, reset='v=v_reset_e', method='euler', name='EG')
        app0['EG'].v = v_rest_e - 20.*mV       
        
        
        app0['ESP'] = SpikeMonitor(app0['EG'], name='ESP')
        app0['ERM'] = PopulationRateMonitor(app0['EG'], name='ERM')                     
        
        # ibhibitory group
        neuron_i = '''
            dv/dt = (ge*(0*mV-v) + (v_rest_i-v)) / (10*ms) : volt
            dge/dt = -ge / (5*ms) : 1
            '''
        app0['IG'] = NeuronGroup(n_i, neuron_i, threshold='v>v_thresh_i', refractory=2*ms, 
                                reset='v=v_reset_i', method='euler', name='IG')
        app0['IG'].v = v_rest_i - 20.*mV
        
        app0['ISP'] = SpikeMonitor(app0['IG'], name='ISP')

        # poisson generators one-to-all excitatory neurons with plastic connections 
        app0['S1'] = Synapses(app0['PG'], app0['EG'], stdp0, on_pre=pre0, on_post=post0, 
                             method='euler', name='S1') 
        app0['S1'].connect()
        app0['S1'].w = 'rand()*gmax' # random weights initialisation
        app0['S1'].lr = 1           # enable stdp    
 
        # excitatory neurons one-to-one inhibitory neurons
        app0['S2'] = Synapses(app0['EG'], app0['IG'], 'w : 1', on_pre='ge += w', name='S2')
        app0['S2'].connect(j='i')
        app0['S2'].delay = 'rand()*10*ms'
        app0['S2'].w = wi # very strong fixed weights to ensure corresponding inhibitory neuron will always fire

        # inhibitory neurons one-to-all-except-one excitatory neurons
        app0['S3'] = Synapses(app0['IG'], app0['EG'], 'w : 1', on_pre='gi += w', name='S3')
        app0['S3'].connect(condition='i!=j')
        app0['S3'].delay = 'rand()*5*ms'
        app0['S3'].w = wi/n_e # weights are selected in such a way as to maintain a balance between excitation and ibhibition

        self.net0 = Network(app0.values())
        self.net0.run(0*second)

    def __getitem__(self, key):
        return self.net[key]

    def train(self, X, targets, epochs=1):
       
        # enable stdp 
        self.net0['S1'].lr = 1         
        
        for ep in range(epochs):
            
            for idx in range(len(X)):
                yt = int(targets[idx]) # target class
         
                # active mode
                self.net0['PG'].rates = X[idx].ravel()*Hz
                self.net0.run(0.35*second)

                # passive mode
                self.net0['PG'].rates = np.zeros(n_input)*Hz
                self.net0.run(0.15*second)

## Model activation

In [6]:
# conditions

train_items = 5000
test_items = 500

In [7]:
model0 = Model0()

## Model training

In [8]:
start = time.time()

model0.train(X_train[:train_items], y_train[:train_items], epochs=1)

end = time.time()

print(f'processing time: {int((end - start)//60)} min {int((end - start)%60)} sec')

processing time: 938 min 41 sec


In [9]:
# saving for later using

model0.net0.store('model0','saved_model0.b2')

# ------------------------------------